In [ ]:
import sys
import os
import enum 
import asyncio
###
from datetime import datetime
from math import floor, ceil, copysign
import time
from time import sleep
###
import json
###
import zmq
zContext = zmq.asyncio.Context()
###
import logging
logger = logging.getLogger()

logger.setLevel(logging.DEBUG)
###
from pymongo import MongoClient
from bson.objectid import ObjectId
import uuid
client = MongoClient()
db=client.data_feeder
###
import ccxt

from dotmap import DotMap

price_ticker = {}
trades = {}


import zmq
import zmq.asyncio
zContext = zmq.asyncio.Context()

In [ ]:
async def run_ticker():
    socket = zContext.socket(zmq.SUB)
    socket.connect("tcp://localhost:5556")
    socket.setsockopt_string(zmq.SUBSCRIBE, 'TICKER')

    while True:
        tickerString = await socket.recv_string()
        # _, exchange, symbol, ticker = tickerString.split()
        print(tickerString)
        # ticker = json.loads(ticker)
        # price_ticker[symbol] = ticker
        # if symbol in trades:
        #     for model in trades[symbol].values():
        #         if model.state in ['LIVE', 'LONG']:
        #             model.tick(ticker=ticker)

from threading import Thread

ticker_loop = asyncio.new_event_loop()

def setup_loop(loop):
    asyncio.set_event_loop(loop)
    loop.run_forever()

ticker_thread = Thread(target=setup_loop, args=(ticker_loop,))
ticker_thread.start()
ticker_future = asyncio.run_coroutine_threadsafe(run_ticker(), ticker_loop)  

load_trades_from_tb()

In [ ]:
def normalize_values(symbol, qty, trigger, stop, stop_limit, target):
    return {'qty': exchange.amount_to_precision(symbol, qty), 
        'trigger': exchange.price_to_precision(symbol, trigger),
        'stop': exchange.price_to_precision(symbol, stop),
        'stop_limit': exchange.price_to_precision(symbol, stop_limit),
        'target': exchange.price_to_precision(symbol, target)
    }


def calc_trade(exchange_name, capital, risk_perc, symbol, trigger, stop, target):
    ntrigger = float(exchange.price_to_precision(symbol, trigger))
    ntarget = float(exchange.price_to_precision(symbol, target))
    nstop = float(exchange.price_to_precision(symbol, stop))
    nstop_limit = float(exchange.price_to_precision(symbol, nstop * (1.0 + copysign(0.05, ntrigger-ntarget))))
    ntarget = float(exchange.price_to_precision(symbol, target))
    risk_size = capital * risk_perc
    nrisk_size = float(exchange.cost_to_precision(symbol, risk_size))

    qty = float(exchange.amount_to_precision(symbol, risk_size / (trigger - stop)))
    size = qty * trigger
    nsize = float(exchange.cost_to_precision(symbol, size))
    proj_profit =float(exchange.cost_to_precision(symbol,(ntarget - ntrigger) * qty))
    proj_loss =float(exchange.cost_to_precision(symbol,(ntrigger - nstop) * qty))
    # profit_perc = round(proj_profit / ntrigger * 100, 2)
    # loss_perc = round(proj_loss / ntrigger * 100, 2)
    r2r = round(proj_profit / proj_loss, 2)
    return {'exchange': exchange_name, 'symbol': symbol, 'size': size, 'risk_size': nrisk_size, 'qty': qty, 'r2r': r2r, 
        'proj_profit': proj_profit, 'proj_loss': proj_loss,
        # 'profit_perc' : profit_perc, 'loss_perc' : loss_perc, 
        'trigger': ntrigger, 'stop': nstop, 'stop_limit': nstop_limit, 'target': ntarget}


def create_trade(exchange_name, capital, risk_perc, symbol, trigger, stop, target):
    exchange = exchanges[exchange_name]
    market = exchange.market(symbol)
    trade_values = calc_trade(exchange_name, capital, risk_perc, symbol, trigger, stop, target)
    trade = TradeModel(exchange, db.trades)
    trade.insert(data={"state": 'DISABLED', 'market': market['id'], **trade_values})
    return trade

def run_trade(exchange_name, capital, risk_perc, symbol, trigger, stop, target):
    trade = create_trade(exchange_name, capital, risk_perc, symbol, trigger, stop, target)
    trade.start()
    watch(trade)
    return trade
